# Notebook for identifying and removing bottlenecks from ICET implementation

In [1]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
from ICET_spherical import ICET
from utils import R_tf
from metpy.calc import lat_lon_grid_deltas

%load_ext autoreload
%autoreload 2
%autosave 180
# %matplotlib notebook

# %%bash
# # python -m cProfile scan_match.py
# python scan_match.py

2022-11-18 15:27:46.066206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-18 15:27:46.151315: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-18 15:27:46.516144: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/cv2/../../lib64:
2022-11-18 15:27:46.516194: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_p

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-11-18 15:27:47.053746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 15:27:47.053978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 15:27:47.054124: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 15:27:47.373199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-18 15:27:47.373385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

Autosaving every 180 seconds


In [2]:
basepath = '/media/derm/06EF-127D1/KITTI'
# sequence = '03' #forest
sequence = '09' #trees and small town
dataset = pykitti.odometry(basepath, sequence)
velo1 = dataset.get_velo(400)
c1 = velo1[:,:3]
velo2 = dataset.get_velo(401)
c2 = velo2[:,:3]

it = ICET(cloud1 = c1, cloud2 = c2, fid = 70, niter = 9, 
           draw = False, group = 2, RM = True, DNN_filter = False)

Ground truth poses are not avaialble for sequence 09.

 loading model took 1.1920928955078125e-06 
 total:  1.0251998901367188e-05

 converting to spherical took 0.05982661247253418 
 total:  0.059891700744628906

 took  0.03638625144958496 seconds to get points in cluster

 getting spherical grid 1.5986287593841553 
 total:  1.6585438251495361

 took  0.01692509651184082 seconds to get points in cluster

 took  0.014208555221557617 seconds to get points in cluster

 took  0.014218330383300781 seconds to get points in cluster

 fit_gaussian for scan 1 0.6818702220916748 
 total:  2.340421438217163

 took  0.03319668769836426 seconds to get points in cluster


2022-11-18 15:27:51.143576: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.



 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.20138192176818848 
 total:  2.542060613632202 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor([ 0.32087612  0.01419324  0.00261957 -0.0005941   0.0014034  -0.00501409], shape=(6,), dtype=float32)

 ~~~~~~~~~~~~~~ 
 correcting solution estimate 0.0724325180053711 
 total:  2.6145617961883545 
 ~~~~~~~~~~~~~~

 took  0.03079676628112793 seconds to get points in cluster


2022-11-18 15:27:51.353436: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x781b230



 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.20845890045166016 
 total:  2.8233001232147217 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor(
[ 6.8456984e-01  1.3711061e-03  8.6955717e-03 -4.5008707e-04
  1.6692462e-03  2.4395338e-03], shape=(6,), dtype=float32)

 ~~~~~~~~~~~~~~ 
 correcting solution estimate 0.020292997360229492 
 total:  2.8436427116394043 
 ~~~~~~~~~~~~~~

 took  0.030968666076660156 seconds to get points in cluster

 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.1984248161315918 
 total:  3.042631149291992 
 ~~~~~~~~~~~~~~

 estimated solution vector X: 
 tf.Tensor(
[ 8.6765587e-01  8.7339478e-03  1.1622415e-02 -2.2419654e-04
  1.9934098e-03  3.0436283e-03], shape=(6,), dtype=float32)

 ~~~~~~~~~~~~~~ 
 correcting solution estimate 0.017867565155029297 
 total:  3.0605571269989014 
 ~~~~~~~~~~~~~~

 took  0.030569076538085938 seconds to get points in cluster

 ~~~~~~~~~~~~~~ 
 fit_gaussian for scan 2 0.19490480422973633 
 total:  3.2557120323181152 
 ~~~~~~

# fit_gaussian()

In [213]:
def fg2(cloud, rag, npts):
    """new method of fitting gaussian to better handle ragged input data"""
    numSamples = 3
    
    coords = tf.gather(cloud, rag)
    mu = tf.math.reduce_mean(coords, axis = 1)[:,None]
    #     print(mu)
    #     print(coords)
    
    xpos = tf.gather(cloud[:,0], rag)
    ypos = tf.gather(cloud[:,1], rag)
    zpos = tf.gather(cloud[:,2], rag)
#     c = tfp.stats.covariance(xpos.to_tensor(), ypos.to_tensor())
    
    #old and slow
#     xx = tf.math.reduce_sum(tf.math.square(xpos - mu[:,:,0] ), axis = 1)/npts

    #TODO: try randomly sampling 30 points from each ragged cell, use reduced num pts to calculate covariance
#     subsampled = tf.map_fn(sample, it.inside2) #works but SLOW
    subsampled = tf.gather(rag,tf.range(tf.shape(rag)[0]))[:numSamples] #wrong
#     print(subsampled)
    
#     nraggeds = tf.shape(coords)[0].numpy()
#     for i in range(nraggeds):
#         c  = tfp.stats.covariance(coords[i])
    
    mu = None
    sigma = None
    return(mu, sigma)

@tf.function
def sample(x, samples=3):
  """https://stackoverflow.com/questions/71073873/sample-from-ragged-tensor"""  
  length = tf.shape(x)[0]
#   was this
#   x = tf.cond(tf.less_equal(length, samples), lambda: x, lambda: tf.gather(x, tf.random.shuffle(tf.range(length))[:samples]))
 
#   test
#   x = tf.cond(tf.less_equal(length, samples), lambda: x, lambda: tf.gather(x, tf.range(length))[:samples])
  x = tf.gather(x,tf.range(length))[:samples]

    
  return x

In [214]:
s = time.time()
mu2, sigma2 = it.fit_gaussian(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print("\n took", time.time() - s, " s with old method")

s = time.time()
mu2, sigma2 = fg2(it.cloud2_tensor, it.inside2, tf.cast(it.npts2, tf.float32))
print(" \n took", time.time() - s, " s with new method")

# print(it.npts2)
# print(it.inside2)


 took 0.1943531036376953  s with old method
<tf.RaggedTensor [[], [], []]>
 
 took 0.010631322860717773  s with new method


In [215]:
# vect = it.inside2
vect = tf.ragged.constant([[1,2,3,4],[5,4,3,2,1],[6],[99],[7,8,9,10,11,12,13]])
# print(tf.shape(vect)[0])
print(vect)
c = tf.map_fn(sample, vect)
print(c)

#wrong
test = tf.gather(vect,tf.range(tf.shape(vect)[0]))[:3]
print(test)


<tf.RaggedTensor [[1, 2, 3, 4], [5, 4, 3, 2, 1], [6], [99], [7, 8, 9, 10, 11, 12, 13]]>
<tf.RaggedTensor [[1, 2, 3], [5, 4, 3], [6], [99], [7, 8, 9]]>
<tf.RaggedTensor [[1, 2, 3, 4], [5, 4, 3, 2, 1], [6]]>
